In [2]:
import numpy as np
import gym
from gym import spaces
import pygame
import torch 

# Import packages
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from typing import List, Tuple
from matplotlib import animation

import collections
from collections import namedtuple, deque
import tqdm
import matplotlib.pyplot as plt
import random
import gymnasium as gym

from IPython.display import clear_output
from IPython import display
from gym.wrappers import FlattenObservation

torch.set_default_dtype(torch.float64)

In [12]:
class baloon1:
    def __init__(self,width,length,height):
        self.x, self.y , self.z = 0,0,0
        self.height = height
        self.width = width
        self.length = length
        self.map = None

    def resest(self,start_x,start_y,start_z):
        self.x, self.y ,self.z = start_x,start_y,start_z

    def up(self):
        self.z = min(self.height,self.z+1)

    def down(self):
        if(self.y>0):
            self.z = max(self.z-1,0)

    def step(self):
        self.x = (self.x + self.map[self.x,self.y,self.z,0])% self.width
        self.y = (self.y + self.map[self.x,self.y,self.z,1])% self.length

    def generate_map(self,eta,mu):
        self.map = np.zeros((self.width,self.length,self.height,2))
        for i in range(self.width):
            for j in  range(self.length):
                for k in range(self.height):
                    for l in range(1):
                        r = np.random.random()
                        if(r>eta):
                            r= np.random.random()
                            if(r>mu):
                                self.map[i,j,k,l] = 1
                            else:
                                self.map[i,j,k,l] = -1


    def set_map(self,map):
        self.map = map

    def get_winds(self,r):
        obs = np.zeros((2*r+1,2*r+1,self.height,2))
        for i in range(self.x - r , self.x + r + 1):
            for j in range(self.y - r, self.y + r + 1):
                for k in range(0,self.height):
                    for l in range(1):
                        obs[j - self.y + r, i-self.x + r]  = (self.map[i%self.width,j%self.length,k,l])
        return obs
    
    def render(self):
        for z in range(self.height):
            for j in range(self.length):
                s = ''
                for i in range(self.width):
                    s+= "[" + str(self.map[i,j,z]) + "]"
                    """if self.x == j and self.y == i : 
                        s+= '+'
                    else:
                        if abs(self.map[i,j,z,1]) == abs(self.map[i,j,z,0]) == 1:
                            s += '/'
                        elif self.map[i,j,z,1] == self.map[i,j,z,0] == 0:
                            s += '.'
                        elif abs(self.map[i,j,z,1]) == 1 and self.map[i,j,z,0] == 0 :
                            s += '|'
                        elif abs(self.map[i,j,z,1]) == 0 and self.map[i,j,z,0] == 1 :
                            s += '-'
                        elif self.map[i,j,z,1] * self.map[i,j,z,0] == -1 :
                            s += '\\'
                print(s)
            s+= '\n'"""
                print(s)
            print(" ")
                
        

In [13]:
bal = baloon1(5,5,3)
bal.generate_map(0.5,0.5)
bal.render()

[[1. 0.]][[0. 0.]][[-1.  0.]][[1. 0.]][[-1.  0.]]
[[-1.  0.]][[0. 0.]][[0. 0.]][[-1.  0.]][[1. 0.]]
[[0. 0.]][[0. 0.]][[-1.  0.]][[0. 0.]][[0. 0.]]
 
[[0. 0.]][[0. 0.]][[0. 0.]][[0. 0.]][[1. 0.]]
[[-1.  0.]][[0. 0.]][[1. 0.]][[0. 0.]][[0. 0.]]
[[-1.  0.]][[-1.  0.]][[-1.  0.]][[0. 0.]][[0. 0.]]
 
[[-1.  0.]][[-1.  0.]][[1. 0.]][[1. 0.]][[0. 0.]]
[[-1.  0.]][[-1.  0.]][[-1.  0.]][[1. 0.]][[-1.  0.]]
[[1. 0.]][[0. 0.]][[1. 0.]][[0. 0.]][[-1.  0.]]
 
[[0. 0.]][[-1.  0.]][[-1.  0.]][[0. 0.]][[0. 0.]]
[[0. 0.]][[0. 0.]][[0. 0.]][[0. 0.]][[1. 0.]]
[[1. 0.]][[1. 0.]][[0. 0.]][[-1.  0.]][[-1.  0.]]
 
[[1. 0.]][[1. 0.]][[0. 0.]][[0. 0.]][[1. 0.]]
[[1. 0.]][[0. 0.]][[0. 0.]][[0. 0.]][[0. 0.]]
[[0. 0.]][[0. 0.]][[1. 0.]][[-1.  0.]][[0. 0.]]
 


In [ ]:
class GridWorldEnv(gym.Env):
    metadata = {"render_modes": ["human", "rgb_array"], "render_fps": 4}
    def __init__(self, render_mode=None, size=10, obs_size = 5 ):
        self.size = size  # The size of the square grid
        self.balloon = baloon1()
        self.obs_size = obs_size

        # Observations are dictionaries with the agent's and the target's location.
        # Each location is encoded as an element of {0, ..., `size`}^2, i.e. MultiDiscrete([size, size]).
        self.observation_space = spaces.Dict(
            {
                "agent": spaces.Box(0, size - 1, shape=(2,), dtype=int),
                "target": spaces.Box(0, size - 1, shape=(2,), dtype=int),
                "map": spaces.Box(-1, 1, shape=(2*obs_size + 1,obs_size*2+1), dtype=int),
            }
        )
        

        self.action_space = spaces.Discrete(3)

    def reset(self):
        self.balloon.generate_map(self.size,self.size , 0.5)

        self.time = 0

        self._agent_location = [2,2]

        self._target_location = [self.size-1,self.size-1]

        observation = self._get_obs()

        return observation, {}

    def step(self, action):
        self.time += 1

        if action == 0:
            self.balloon.step()
        if action == 1:
            self.balloon.up()
        if action == 2:
            self.balloon.down()

        self._agent_location = np.array([self.balloon.x , self.balloon.y])
        terminated = np.array_equal(self._agent_location, self._target_location)
        reward = -(self.time + np.sqrt((self._agent_location[0] - self._target_location[0])**2 + (self._agent_location[1] - self._target_location[1])**2))
        
        observation = self._get_obs()

        return observation, reward, terminated, False, {}
    
    def _get_obs(self):
        return {"agent": self._agent_location, "target": self._target_location , "map" : self.balloon.get_winds(self.obs_size)}

    def render(self):
        self.balloon.render()
        
    def close(self):
        pass

In [ ]:
class FullyConnectedModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(FullyConnectedModel, self).__init__()
        self.soft = torch.nn.Softmax(dim = -1)
        # Define layers with ReLU activation
        self.linear1 = nn.Linear(input_size, 30)
        self.activation1 = nn.Sigmoid()
        self.linear2 = nn.Linear(30, 30)
        self.activation2 = nn.Sigmoid()
        self.linear3 = nn.Linear(30, 30)
        self.activation3 = nn.Sigmoid()

        # Output layer without activation function
        self.output_layer = nn.Linear(30, output_size)

        # Initialization using Xavier uniform (a popular technique for initializing weights in NNs)
        nn.init.xavier_normal_(self.linear1.weight)
        nn.init.xavier_normal_(self.linear2.weight)
        nn.init.xavier_normal_(self.linear3.weight)
        nn.init.xavier_normal_(self.output_layer.weight)

    def forward(self, inputs):
        # Forward pass through the layers
        x = self.activation1(self.linear1(inputs))
        x = self.activation2(self.linear2(x))
        x = self.activation3(self.linear3(x))
        x = self.soft(self.output_layer(x))
        return x

    
class QNetwork:
    def __init__(self, env, lr, logdir=None):
        # Define Q-network with specified architecture
        self.net = FullyConnectedModel(4, 2)
        self.env = env
        self.lr = lr 
        self.logdir = logdir
        self.optimizer = optim.Adam(self.net.parameters(), lr=self.lr)

    def load_model(self, model_file):
        # Load pre-trained model from a file
        return self.net.load_state_dict(torch.load(model_file))

    def load_model_weights(self, weight_file):
        # Load pre-trained model weights from a file
        return self.net.load_state_dict(torch.load(weight_file))
    
class QPolicy:
    def __init__(self, s_size, a_size):
        self.net = FullyConnectedModel(s_size,a_size)

    def act(self, state):
        state = torch.tensor(state)
        probs = self.net(state)
        m = torch.distributions.Categorical(probs)
        action = m.sample()
        return action.item(), m.log_prob(action)

In [ ]:
def reinforce(policy, optimizer, n_training_episodes, max_t, gamma, print_every,env):
    scores_deque = deque(maxlen=100)
    scores = []
    for i_episode in range(1, n_training_episodes + 1):
        saved_log_probs = []
        rewards = []
        state, _ = env.reset()
        for t in range(max_t):
            action, log_prob = policy.act(torch.tensor(state,dtype=torch.double))
            saved_log_probs.append(log_prob)
            state, reward, done, truncated , _ = env.step(action)
            rewards.append(reward)
            if done or truncated:
                break
        scores_deque.append(sum(rewards))
        scores.append(sum(rewards))

        returns = deque(maxlen=max_t)
        n_steps = len(rewards)
        # Compute the discounted returns at each timestep,
        # as
        #      the sum of the gamma-discounted return at time t (G_t) + the reward at time t
        #
        # In O(N) time, where N is the number of time steps
        # (this definition of the discounted return G_t follows the definition of this quantity
        # shown at page 44 of Sutton&Barto 2017 2nd draft)
        # G_t = r_(t+1) + r_(t+2) + ...

        # Given this formulation, the returns at each timestep t can be computed
        # by re-using the computed future returns G_(t+1) to compute the current return G_t
        # G_t = r_(t+1) + gamma*G_(t+1)
        # G_(t-1) = r_t + gamma* G_t
        # (this follows a dynamic programming approach, with which we memorize solutions in order
        # to avoid computing them multiple times)

        # This is correct since the above is equivalent to (see also page 46 of Sutton&Barto 2017 2nd draft)
        # G_(t-1) = r_t + gamma*r_(t+1) + gamma*gamma*r_(t+2) + ...

        ## Given the above, we calculate the returns at timestep t as:
        #               gamma[t] * return[t] + reward[t]
        #
        ## We compute this starting from the last timestep to the first, in order
        ## to employ the formula presented above and avoid redundant computations that would be needed
        ## if we were to do it from first to last.

        ## Hence, the queue "returns" will hold the returns in chronological order, from t=0 to t=n_steps
        ## thanks to the appendleft() function which allows to append to the position 0 in constant time O(1)
        ## a normal python list would instead require O(N) to do this.
        for t in range(n_steps)[::-1]:
            disc_return_t = returns[0] if len(returns) > 0 else 0
            returns.appendleft(gamma * disc_return_t + rewards[t])

        ## standardization of the returns is employed to make training more stable
        eps = np.finfo(np.float32).eps.item()
        ## eps is the smallest representable float, which is
        # added to the standard deviation of the returns to avoid numerical instabilities
        returns = torch.tensor(returns)
        returns = (returns - returns.mean()) / (returns.std() + eps)

        # Line 7:
        policy_loss = 0
        for log_prob, disc_return in zip(saved_log_probs, returns):
            policy_loss += (-log_prob * disc_return)

        # Line 8: PyTorch prefers gradient descent
        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()
        #print(policy.net.linear3.weight)

        if i_episode % print_every == 0:
            print("Episode {}\tAverage Score: {:.2f}".format(i_episode, np.mean(scores_deque)))

    return scores

In [ ]:
# Create policy and place it to the device

from gym.wrappers import FlattenObservation



hyperparameters = {
    "n_training_episodes": 1000,
    "n_evaluation_episodes": 10,
    "max_t": 10,
    "gamma": 1.0,
    "lr": 1e-7,
    "grid_size" : 5,
    "obs_r" :3,
    "action_space": 3,
    "print" : 1
}

policy = QPolicy(
    (hyperparameters["obs_r"]*2+1)**2 + 4,hyperparameters["action_space"]
)
optimizer = optim.Adam(policy.net.parameters(), lr=hyperparameters["lr"])

In [ ]:
env = GridWorldEnv(size = hyperparameters["grid_size"], obs_size=hyperparameters["obs_r"])
wrapped_env = FlattenObservation(env)

scores = reinforce(
    policy,
    optimizer,
    hyperparameters["n_training_episodes"],
    hyperparameters["max_t"],
    hyperparameters["gamma"],
    hyperparameters["print"],
    wrapped_env)

C:\Users\monce\AppData\Local\Temp\ipykernel_28132\1435207334.py:53: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state = torch.tensor(state)


Episode 1	Average Score: -94.22
Episode 2	Average Score: -95.27
Episode 3	Average Score: -91.75
Episode 4	Average Score: -93.22
Episode 5	Average Score: -93.84
Episode 6	Average Score: -93.25
Episode 7	Average Score: -94.11
Episode 8	Average Score: -95.37
Episode 9	Average Score: -95.62
Episode 10	Average Score: -94.59
Episode 11	Average Score: -94.17
Episode 12	Average Score: -93.73
Episode 13	Average Score: -94.64
Episode 14	Average Score: -95.38
Episode 15	Average Score: -94.82
Episode 16	Average Score: -94.56
Episode 17	Average Score: -94.10
Episode 18	Average Score: -93.58
Episode 19	Average Score: -93.41
Episode 20	Average Score: -92.29
Episode 21	Average Score: -91.70
Episode 22	Average Score: -90.89
Episode 23	Average Score: -90.75
Episode 24	Average Score: -88.65
Episode 25	Average Score: -85.14


C:\Users\monce\AppData\Local\Temp\ipykernel_28132\450940638.py:58: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ..\aten\src\ATen\native\ReduceOps.cpp:1760.)
  returns = (returns - returns.mean()) / (returns.std() + eps)


ValueError: Expected parameter probs (Tensor of shape (3,)) of distribution Categorical(probs: torch.Size([3])) to satisfy the constraint Simplex(), but found invalid values:
tensor([nan, nan, nan], grad_fn=<DivBackward0>)

In [ ]:

state, _ = wrapped_env.reset()
print(policy.net.linear1.weight)
(torch.tensor(state,dtype=torch.double))


NameError: name 'wrapped_env' is not defined

In [ ]:
env2 = GridWorldEnv(size = hyperparameters["grid_size"], obs_size=hyperparameters["obs_r"])
env2.reset()
env2.render()
for i in range(5):
    state, reward, done, truncated , _ =env2.step(env2.action_space.sample())
    env2.render()
    print(" ")
    print(reward)
    print(" ")

+><><
<<><<
><<>>
><<>>
<<>><

>+<><
<<><<
><<>>
><<>>
<<>><

 
-6.0
 
>+<><
<<><<
><<>>
><<>>
<<>><

 
-7.0
 
>+<><
<<><<
><<>>
><<>>
<<>><

 
-8.0
 
>+<><
<<><<
><<>>
><<>>
<<>><

 
-9.0
 
>><><
<-><<
><<>>
><<>>
<<>><

 
-9.242640687119284
 


In [ ]:
env2.action_space.sample()

2